In [1]:
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install mltu==1.0.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flatbuffers-23.5.26
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0
  Attempting uninstall: libr

In [4]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

from mltu.preprocessors import ImageReader
from mltu.transformers import ImageResizer, LabelIndexer, LabelPadding, ImageShowCV2
from mltu.augmentors import RandomBrightness, RandomRotate, RandomErodeDilate, RandomSharpen
from mltu.annotations.images import CVImage

from mltu.tensorflow.dataProvider import DataProvider
from mltu.tensorflow.losses import CTCloss
from mltu.tensorflow.callbacks import Model2onnx, TrainLogger
from mltu.tensorflow.metrics import CERMetric, WERMetric

import os
from tqdm import tqdm

In [5]:
# %cd drive/MyDrive/HSR\ -\ Chocosy
!ls
!unzip "/content/drive/MyDrive/AIT301 - HSR/data/sentences.zip" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/sentences/h07/h07-025a/h07-025a-00.png  
  inflating: /content/sentences/h07/h07-025a/h07-025a-01.png  
  inflating: /content/sentences/h07/h07-025a/h07-025a-02.png  
  inflating: /content/sentences/h07/h07-025a/h07-025a-03.png  
  inflating: /content/sentences/h07/h07-025a/h07-025a-04.png  
  inflating: /content/sentences/h07/h07-025a/h07-025a-05.png  
  inflating: /content/sentences/h07/h07-025a/h07-025a-06.png  
  inflating: /content/sentences/h07/h07-025a/h07-025a-07.png  
  inflating: /content/sentences/h07/h07-025a/h07-025a-08.png  
 extracting: /content/sentences/h07/h07-025a/h07-025a-09.png  
   creating: /content/sentences/h07/h07-028/
  inflating: /content/sentences/h07/h07-028/h07-028-00.png  
  inflating: /content/sentences/h07/h07-028/h07-028-01.png  
  inflating: /content/sentences/h07/h07-028/h07-028-02.png  
  inflating: /content/sentences/h07/h07-028/h07-028-03.png  
  inflating: /content/sentence

In [6]:
# %cd
data_label_path = "/content/drive/MyDrive/AIT301 - HSR/data/lines.txt"
data_folder_path = "/content/sentences"

In [7]:
dataset, vocab, max_len = [], set(), 0
words = open(data_label_path, "r").readlines()

for line in tqdm(words):
    # if len(dataset) == 2000:
    #   break

    if line.startswith('#'):
        continue

    line_split = line.split(" ")

    #remember to calculate the number of err lines to compare with the total numbers of sentences in the dataset
    if line_split[2] == 'err':
        continue

    folder1 = line_split[0][:3]
    folder2 = '-'.join(line_split[0].split("-")[:2])
    file_name = line_split[0] + ".png"
    label = line_split[-1].rstrip('\n')

    label = label.replace('|', " ")

    relative_path = os.path.join(data_folder_path, folder1, folder2, file_name)

    if not os.path.exists(relative_path):
        print(f"File not found: {relative_path}")
        continue

    dataset.append([relative_path, label])

    # vocab.update(list(label))
    # max_len = max(max_len, len(label))

100%|██████████| 13376/13376 [00:00<00:00, 82188.73it/s]


In [21]:
import random

# random.seed(0) #make sure every shuffle of dataset is the same
# dataset = random.sample(dataset, len(dataset))[:2500] #decrease the size of dataset
dataset = dataset[:2500]
for path, label in dataset:
    vocab.update(list(label))
    max_len = max(max_len, len(label))
print(len(dataset))
print(len(vocab))

2500
77


In [9]:
import os
from datetime import datetime

from mltu.configs import BaseModelConfigs

class MyModelConfigurations(BaseModelConfigs):
    def __init__(self):
        super().__init__()

        self.model_path = os.path.join("Models/Sentence_recognition", datetime.strftime(datetime.now(), "%Y%m%d%H%M"))
        self.vocab = ""
        self.height = 96
        self.width = 1408
        self.max_text_length = 0
        self.batch_size = 32
        self.learning_rate = 0.0005
        self.train_epochs = 100
        self.train_workers = 20

In [26]:
configs = MyModelConfigurations()

# tmp = "E/R9Cd?3F#vMhz*rISVOs0n1o7f(mwLe-QPt,:pDuH4G\"g!WZ5jNy8kxYJqX\'lcU)A;2.bTKB6 ai"
# print(len(tmp))
# print(len(vocab))
print(max_len)

configs.vocab = "".join(tmp)
configs.max_text_length = max_len
configs.save()

77
77
74


In [27]:
data_provider = DataProvider(
    dataset = dataset,
    skip_validation = True,
    batch_size = configs.batch_size,
    data_preprocessors = [ImageReader(CVImage)],
    transformers = [
        ImageResizer(configs.width, configs.height, keep_aspect_ratio = True),
        LabelIndexer(configs.vocab),
        LabelPadding(max_word_length = configs.max_text_length, padding_value = len(configs.vocab))
    ]
)

train_data_provider, validate_data_provider = data_provider.split(split = 0.9)
train_data_provider.augmentors = [
    RandomBrightness(),
    RandomErodeDilate(),
    RandomSharpen()
]

In [1]:
from keras import layers
from keras.models import Model

from mltu.tensorflow.model_utils import residual_block


def train_model(input_dim, output_dim, activation="leaky_relu", dropout=0.2):

    inputs = layers.Input(shape=input_dim, name="input")

    # normalize images here instead in preprocessing step
    input = layers.Lambda(lambda x: x / 255)(inputs)

    x1 = residual_block(input, 32, activation=activation, skip_conv=True, strides=1, dropout=dropout)

    x2 = residual_block(x1, 32, activation=activation, skip_conv=True, strides=2, dropout=dropout)
    x3 = residual_block(x2, 32, activation=activation, skip_conv=False, strides=1, dropout=dropout)

    x4 = residual_block(x3, 64, activation=activation, skip_conv=True, strides=2, dropout=dropout)
    x5 = residual_block(x4, 64, activation=activation, skip_conv=False, strides=1, dropout=dropout)

    x6 = residual_block(x5, 128, activation=activation, skip_conv=True, strides=2, dropout=dropout)
    x7 = residual_block(x6, 128, activation=activation, skip_conv=True, strides=1, dropout=dropout)

    x8 = residual_block(x7, 128, activation=activation, skip_conv=True, strides=2, dropout=dropout)
    x9 = residual_block(x8, 128, activation=activation, skip_conv=False, strides=1, dropout=dropout)

    squeezed = layers.Reshape((x9.shape[-3] * x9.shape[-2], x9.shape[-1]))(x9)

    blstm = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(squeezed)
    blstm = layers.Dropout(dropout)(blstm)

    # blstm = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(squeezed)
    # blstm = layers.Dropout(dropout)(blstm)

    blstm = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(blstm)
    blstm = layers.Dropout(dropout)(blstm)

    output = layers.Dense(output_dim + 1, activation="softmax", name="output")(blstm)

    model = Model(inputs=inputs, outputs=output)
    return model

ModuleNotFoundError: No module named 'keras'

In [29]:
model = train_model(input_dim = (configs.height, configs.width, 3),
                    output_dim = len(configs.vocab))

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = configs.learning_rate),
    loss = CTCloss(),
    metrics = [
        CERMetric(vocabulary = configs.vocab),
        WERMetric(vocabulary = configs.vocab)
    ],
    run_eagerly = False)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 96, 1408, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda_1 (Lambda)              (None, 96, 1408, 3)  0           ['input[0][0]']                  
                                                                                                  
 conv2d_24 (Conv2D)             (None, 96, 1408, 32  896         ['lambda_1[0][0]']               
                                )                                                                 
                                                                                            

In [30]:
callbacks = [
    # EarlyStopping(monitor = "val_CER", patience = 20, verbose = 1, mode = "min"),
    ModelCheckpoint(f"{configs.model_path}/model.h5", monitor = "val_CER", verbose = 1, save_best_only = True, mode = "min"),
    TrainLogger(configs.model_path),
    TensorBoard(f"{configs.model_path}/logs", update_freq = 1),
    ReduceLROnPlateau(monitor = "val_CER", factor = 0.9, min_delta = 1e-10, patience = 5, verbose = 1, mode = "auto"),
    Model2onnx(f"{configs.model_path}/model.h5")
]

In [ ]:
model.fit(
    train_data_provider,
    validation_data = validate_data_provider,
    epochs = 180,
    callbacks = callbacks,
    workers = configs.train_workers
)

train_data_provider.to_csv(os.path.join(configs.model_path, "train.csv"))
validate_data_provider.to_csv(os.path.join(configs.model_path, "validate.csv"))

In [31]:
model.load_weights("/content/model.h5")

In [32]:
import pandas as pd
from tqdm import tqdm
from mltu.configs import BaseModelConfigs
import cv2
import typing
import numpy as np

from mltu.inferenceModel import OnnxInferenceModel
from mltu.utils.text_utils import ctc_decoder, get_cer, get_wer
from mltu.transformers import ImageResizer

# print(configs.vocab)

# test_model = ImageToWordModel("model.onnx", char_list=configs.vocab)

# df = pd.read_csv("/content/Models/Sentence_recognition/202306030938/validate.csv").values.tolist()


def predict(image, model):
        image = ImageResizer.resize_maintaining_aspect_ratio(image, 1408, 96)

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = model.predict(image_pred)

        text = ctc_decoder(preds, configs.vocab)[0]

        return text

df = pd.read_csv("/content/validate.csv").values.tolist()

# df = pd.read_csv("/content/Models/Sentence_recognition/202306270026/validate.csv").values.tolist()


cnt = 0

accum_cer, accum_wer = [], []
for image_path, label in tqdm(df):
    # if cnt == 1:
    #   break
    cnt += 1
    print(image_path, label)

    # image_path = image_path.split("/")
    # image_path[1] = 'data'
    # image_path = '/'.join(image_path[1:])

    image = np.asarray(cv2.imread(image_path))

    prediction_text = predict(image, model)
    # model.predict(image)

    cer = get_cer(prediction_text, label)
    wer = get_wer(prediction_text, label)
    print("Image: ", image_path)
    print("Label:", label)
    print("Prediction: ", prediction_text)
    print(f"CER: {cer}; WER: {wer}")

    accum_cer.append(cer)
    accum_wer.append(wer)
    print(prediction_text, label)

print(f"Average CER: {np.average(accum_cer)}, Average WER: {np.average(accum_wer)}")

  0%|          | 0/250 [00:00<?, ?it/s]

/content/sentences/a01/a01-132/a01-132-05.png claimed was underpinning - not
1/1 [==============================] - 2s 2s/step


  0%|          | 1/250 [00:01<07:37,  1.84s/it]

Image:  /content/sentences/a01/a01-132/a01-132-05.png
Label: claimed was underpinning - not
Prediction:  chainned awas undupinming - not
CER: 0.23333333333333334; WER: 0.6
chainned awas undupinming - not claimed was underpinning - not
/content/sentences/a01/a01-077u/a01-077u-06.png He said he was waiting for his senior
1/1 [==============================] - 0s 65ms/step


  1%|          | 2/250 [00:01<03:25,  1.20it/s]

...

Image:  /content/sentences/a05/a05-000/a05-000-01.png
Label: be published in March , is likely to
Prediction:  be pubished in March , is likely to
CER: 0.027777777777777776; WER: 0.125
be pubished in March , is likely to be published in March , is likely to
/content/sentences/a01/a01-020u/a01-020u-04.png attending . These plans do not give the
1/1 [==============================] - 0s 67ms/step


100%|█████████▉| 249/250 [00:30<00:00,  8.29it/s]

Image:  /content/sentences/a01/a01-020u/a01-020u-04.png
Label: attending . These plans do not give the
Prediction:  attending . These plous do not give the
CER: 0.05128205128205128; WER: 0.125
attending . These plous do not give the attending . These plans do not give the
/content/sentences/b04/b04-103/b04-103-01.png they would never behave like that . But do they ever
1/1 [==============================] - 0s 64ms/step


100%|██████████| 250/250 [00:30<00:00,  8.15it/s]

Image:  /content/sentences/b04/b04-103/b04-103-01.png
Label: they would never behave like that . But do they ever
Prediction:  Hhey wald neser belare the that. Bot do thy aser
CER: 0.2692307692307692; WER: 0.9090909090909091
Hhey wald neser belare the that. Bot do thy aser they would never behave like that . But do they ever
Average CER: 0.11166490924597065, Average WER: 0.35269169719169724


In [34]:

# image_path = "/content/sentences/r03/r03-056/r03-056-01.png"
image_path = "/content/test2.png"
# image_path = image_path.split("/")
# image_path[1] = 'data'
# image_path = '/'.join(image_path[1:])

image = np.asarray(cv2.imread(image_path))
label = "people are bird"

prediction_text = predict(image, model)
# model.predict(image)

cer = get_cer(prediction_text, label)
wer = get_wer(prediction_text, label)
print("Image: ", image_path)
# print("Label:", label)
print("Prediction: ", prediction_text)
print(f"CER: {cer}; WER: {wer}")

# print(prediction_text, label)

1/1 [==============================] - 0s 63ms/step
Image:  /content/test2.png
Prediction:  people are bird 
CER: 0.06666666666666667; WER: 0.0
